In [56]:
import pandas as pd, numpy as np, glob

# Change the settings to output thousand separators: Use f'{value:n}' For Python ≥3.6

# Set the path where the data is saved. The code will pick all csv files
path = r'C:/Users/jose.delapaznoguera/Projects/NRAS/Data'
filenames = glob.glob(path + "/*.csv")

# Create an temporary list to store the content of each file
my_list = []
for filename in filenames:
    my_list.append(pd.read_csv(filename))

# Transform the list into a dataframe
my_df = pd.concat(my_list, ignore_index=True)
my_df.columns = [c.replace(' ','_') for c in my_df.columns]

# Make sure we created a dataframe
#print(type(my_df))

# Print the first few rows to take a look. Tick the option to show all the columns
pd.set_option('display.max_columns', None)
#print(my_df.head(15))

# Take a look at the current totals per category
print(my_df.groupby("Origin_Category").Origin_Category.count())
print(my_df.groupby("Destination_Category").Destination_Category.count())

# Replace alphabetical categories by a numerical score based on a dictionary
numeric_df = my_df
my_dict = {'0': 0,'A': 1, 'B': 2, 'B1': 3, 'B2': 4, 'B3': 5, 'C': 6, 'Null': -1}
numeric_df.Origin_Category = numeric_df.Origin_Category.map(my_dict)
numeric_df.Destination_Category = numeric_df.Destination_Category.map(my_dict)

# Remove OD pairs where the score is 0 or Null. Save the number of removed records for traceability
dropped_origins = numeric_df.Origin_Category[numeric_df.Origin_Category < 1].count()
numeric_df = numeric_df.drop(numeric_df[numeric_df.Origin_Category < 1].index)
dropped_destinations = numeric_df.Destination_Category[numeric_df.Destination_Category < 1].count()
numeric_df = numeric_df.drop(numeric_df[numeric_df.Destination_Category < 1].index)
#numeric_df.info()

print(f"A total of {dropped_origins:n} origins and {dropped_destinations:n} destinations were invalid and not included in the analysis.")

# Categorize the journeys by looking at the maximum numerical score of the origin and destination stations
jny_Category = np.maximum(numeric_df.Origin_Category, numeric_df.Destination_Category)

# Add the list as a new column to the existing dataframe
numeric_df['jny_Category'] = jny_Category

#numeric_df.info()
v1 = numeric_df.groupby("jny_Category").Total_Journeys.sum()
print(v1)
print(type(v1))
#print(numeric_df.tail(10))


Origin_Category
0         1127
A       635030
B        42884
B1      285288
B2      203125
B3      132530
C       120235
Null      9545
Name: Origin_Category, dtype: int64
Destination_Category
0         1121
A       627688
B        44701
B1      286359
B2      200799
B3      132907
C       124480
Null     11709
Name: Destination_Category, dtype: int64
A total of 10,672 origins and 12,733 destinations were invalid and not included in the analysis.
jny_Category
1    611985670
2     10656558
3    241547256
4    170671927
5    114344146
6    175730708
Name: Total_Journeys, dtype: int64
<class 'pandas.core.series.Series'>
